In [13]:
using Random
using SCS
using Test
using BenchmarkTools
using MathOptInterface
using SparseArrays
using MatrixOptInterface
using DiffOpt

const MOI  = MathOptInterface
const MatOI = MatrixOptInterface
const MOIU = MathOptInterface.Utilities;

const ATOL=1e-3
const RTOL=1e-3;

In [14]:
# starting w/ a simple LP

D = 50  # variable dimension
M = 50  # no of equality constraints
N = 60; # no of inequality constraints

In [15]:
x̂ = rand(D) # solution

c = rand(D) # objective coeffs

A = rand(M, D) # equality part
b = A*x̂

G = rand(N, D) # inequality part
h = G*x̂ .+ rand(N);

In [16]:
model = diff_optimizer(SCS.Optimizer)
x = MOI.add_variables(model, D)

# define objective
objective_function = MOI.ScalarAffineFunction(MOI.ScalarAffineTerm.(c, x), 0.0)
MOI.set(model, MOI.ObjectiveFunction{MOI.ScalarAffineFunction{Float64}}(), objective_function)
MOI.set(model, MOI.ObjectiveSense(), MOI.MIN_SENSE)

# set equality constraints
# # push!(constraint_indices, MOI.add_constraint(model,MOI.ScalarAffineFunction(MOI.ScalarAffineTerm.(A[i,:], x), 0.),MOI.LessThan(b[i])))
ceq = [ 
    MOI.add_constraint(
        model,
        MOI.VectorAffineFunction(
            MOI.VectorAffineTerm.(1:1, MOI.ScalarAffineTerm.(A[i,:], x)),
            [-b[i]]
        ),
        MOI.Zeros(1)
    ) for i in 1:M   
]

# set inequality constraints
cineq = [ 
    MOI.add_constraint(
        model,
        MOI.VectorAffineFunction(
            MOI.VectorAffineTerm.(1:1, MOI.ScalarAffineTerm.(-G[i,:], x)),
            [h[i]]
        ),
        MOI.Nonnegatives(1)
    ) for i in 1:N   
]

MOI.set(model, MOI.Silent(), true)
MOI.optimize!(model)

x̄ = MOI.get(model, MOI.VariablePrimal(), x);  # solution

In [17]:
# testing solution validity
@test (c'x̄ <= c'x̂) || c' * (x̄ - x̂) <= ATOL  
@test A*x̄ ≈ b atol=ATOL rtol=RTOL
@test G*x̄ <= h

Test Passed

In [18]:
# dA = zeros(11,5)
db = zeros(110)
dc = zeros(50);

In [20]:
@benchmark backward_conic!($model, dA, $db, $dc) setup=(dA=rand(110,50))

BenchmarkTools.Trial: 
  memory estimate:  151.44 MiB
  allocs estimate:  187285
  --------------
  minimum time:     117.138 ms (14.92% GC)
  median time:      130.666 ms (15.70% GC)
  mean time:        137.346 ms (18.12% GC)
  maximum time:     275.103 ms (64.13% GC)
  --------------
  samples:          37
  evals/sample:     1

In [28]:
# # might slow down computation
# # need to find a faster way
# function CSRToCSC(B::MatOI.SparseMatrixCSRtoCSC{Int64})
#     A = sparse(zeros(B.m, B.n))
#     last = 0
#     for i in 1:B.n
#         rnge = (last+1):B.colptr[i]
#         A[(1 .+ B.rowval[rnge]), i] = B.nzval[rnge]
#         last = B.colptr[i]
#     end
#     return A
# end

# conic_form = MatOI.get_conic_form(Float64, model.optimizer, model.con_idx)
# # print(Array(CSRToCSC(conic_form.A)))
# # print(conic_form.b)
# # print(conic_form.c)